# <h1 align=center> **ETL - DATASET CREDITS** </h1>
<h1 align=center> (Extract, Transform, Load) </h1>

Como fase previa a la aplicación del ETL, se realizará la importación de librerías que serán de utilidad en el desarrollo del proceso, las cuales son:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
from scipy.spatial import distance
import json
import ast
import os
import re

# <h1 align=left>**`Extract`**</h1>

In [2]:
df_credits = pd.read_csv("..\Data\credits.csv")

In [3]:
df_movies = pd.read_csv("..\Data\movies_dataset_ETL.csv")

# <h1 align=left>**`Transform`**</h1>

Se consulta los datos que contiene el DataFrame, para identificar su estructura y la información que contiene y se procede a reorganizar las variables para que el **`id`** quede en primera posición.

In [ ]:
df_credits = df_credits.reindex(columns=['id',
                                        'crew',
                                        'cast'])                                        
df_credits

**1.** Se eliminan las comillas simples dentro de comillas dobles. 

In [5]:
def remove_single_quotes_within_double_quotes(text):
    pattern = r'"([^"]*?\'[^"]*?)"'
    matches = re.findall(pattern, text)
    for match in matches:
        text = text.replace(match, match.replace("'", " "))
    return text

**2.** Aplicar la función remove_single_quotes_within_double_quotes() al DataFrame.

In [6]:
df_credits["crew"] = df_credits["crew"].apply(remove_single_quotes_within_double_quotes)

**3.** Se realiza limpieza a la variable **`id`** y en general al DataFrame, así:

+ Convertimos la columna "id" a entero

+ Eliminar duplicados en la columna "id"

+ Reorganizar las columnas del dataframe

+ Reemplazar caracteres ' por " en la columna: "crew"

+ Reemplazar texto "None" por "" (espacios vacíos) en la columna "crew" para solucionar errores del formato Json

+ Organizar columna por "id" (ascendente)

In [7]:
df_credits = df_credits.astype({"id": "int64"})

df_credits = df_credits.drop_duplicates(subset=["id"])

df_credits = df_credits[["id", "cast", "crew"]]

df_credits["crew"] = df_credits["crew"].str.replace("'", "\"", case=False, regex=False)

df_credits["crew"] = df_credits["crew"].str.replace("None", "\"\"", case=False, regex=False)

df_credits = df_credits.sort_values(["id"], ignore_index=True)

**4.** Se crea la función **`extract_director_name`** con la librería **_`json`_** la cual extrae de la variable **`crew`** todos los nombres de las personas asignadas con trabajo 'Director' y la fórmula se aplica en una nueva variable llamada **`drection_name`**.

In [8]:
def extract_director_name(crew_json):
    try:
        crew_list = json.loads(crew_json)
        director_names = [entry['name'] for entry in crew_list if entry['job'] == 'Director']
        return director_names[0] if len(director_names)== 1 else director_names
    except (json.JSONDecodeError, TypeError):
        return None

df_credits["direction_name"] = df_credits["crew"].apply(extract_director_name)

**5.** Se consulta el DataFrame para validar la correcta aplicación de la fórmula.

In [9]:
df_credits.head(3)

,id,cast,crew,direction_name
0,2,"[{'cast_id': 3, 'character': 'Taisto Olavi Kas...","[{""credit_id"": ""52fe420dc3a36847f800001f"", ""de...",Aki Kaurismäki
1,3,"[{'cast_id': 5, 'character': 'Nikander', 'cred...","[{""credit_id"": ""52fe420dc3a36847f8000077"", ""de...",Aki Kaurismäki
2,5,"[{'cast_id': 42, 'character': 'Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800011b"", ""de...","[Allison Anders, Alexandre Rockwell, Robert Ro..."


**6.** Se unen los dos DataFrames importados a través de la función **`.merge`**.

In [10]:
df_movies_crew = pd.merge(df_movies, df_credits[['id', 'direction_name']], on='id', how='left')
df_movies_crew.head(10)

,Unnamed: 0,id,title,overview,tagline,id_btc,name_btc,release_date,release_year,budget,...,country_name,original_language,languages_code,languages_name,runtime,vote_average,popularity,vote_count,status,direction_name
0,45463,0,Midnight Man,0,NaN,NaN,NaN,1997-08-20,1997,0.0,...,NaN,0,en,English,104.0,6.0,0.065736,1.0,Released,NaN
1,45464,1,Mardock Scramble: The Third Exhaust,0,NaN,NaN,NaN,2012-09-29,2012,0.0,...,NaN,0,ja,日本語,68.0,7.0,1.931659,12.0,Released,NaN
2,4342,2,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,NaN,NaN,NaN,1988-10-21,1988,0.0,...,NaN,fi,"fi, de","suomi, Deutsch",69.0,7.1,3.860491,44.0,Released,Aki Kaurismäki
3,12947,3,Shadows in Paradise,"An episode in the life of Nikander, a garbage ...",NaN,NaN,NaN,1986-10-16,1986,0.0,...,NaN,fi,"en, fi, sv","English, suomi, svenska",76.0,7.1,2.292110,35.0,Released,Aki Kaurismäki
4,45465,4,Avalanche Sharks,0,Beware Of Frost Bites,NaN,NaN,2014-01-01,2014,0.0,...,NaN,0,en,English,82.0,4.3,2.185485,22.0,Released,NaN
5,17,5,Four Rooms,It's Ted the Bellhop's first night on the job....,Twelve outrageous guests. Four scandalous requ...,NaN,NaN,1995-12-09,1995,4000000.0,...,NaN,en,en,English,98.0,6.5,9.026586,539.0,Released,"[Allison Anders, Alexandre Rockwell, Robert Ro..."
6,474,6,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",Don't move. Don't whisper. Don't even breathe.,NaN,NaN,1993-10-15,1993,0.0,...,NaN,en,en,English,110.0,6.4,5.538671,79.0,Released,Stephen Hopkins
7,256,11,Star Wars,Princess Leia is captured and held hostage by ...,"A long time ago in a galaxy far, far away...",10.0,Star Wars Collection,1977-05-25,1977,11000000.0,...,NaN,en,en,English,121.0,8.1,42.149697,6778.0,Released,George Lucas
8,6232,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",There are 3.7 trillion fish in the ocean. They...,137697.0,Finding Nemo Collection,2003-05-30,2003,94000000.0,...,NaN,en,en,English,100.0,7.6,25.497794,6292.0,Released,Andrew Stanton
9,351,13,Forrest Gump,A man with a low IQ has accomplished great thi...,"The world will never be the same, once you've ...",NaN,NaN,1994-07-06,1994,55000000.0,...,NaN,en,en,English,142.0,8.2,48.307194,8147.0,Released,Robert Zemeckis


**7.** Se realiza la consulta del nombre de todas las variables del DataFrame.

In [11]:
columns= df_movies_crew.columns.tolist()
columns

['Unnamed: 0',
 'id',
 'title',
 'overview',
 'tagline',
 'id_btc',
 'name_btc',
 'release_date',
 'release_year',
 'budget',
 'revenue',
 'return',
 'genres_id',
 'genres_name',
 'ption_companies_id',
 'ption_companies_name',
 'country_code',
 'country_name',
 'original_language',
 'languages_code',
 'languages_name',
 'runtime',
 'vote_average',
 'popularity',
 'vote_count',
 'status',
 'direction_name']

**8.** Se crea un nuevo DataFrame con las columnas deseadas

In [12]:
columns_newdf = ['id',
                'title',
                'direction_name',
                'release_date',
                'budget',
                'revenue',
                'return']
df_director = df_movies_crew[columns_newdf].copy()

+ Se consulta las filas iniciales del nuevo DataFrame para falidar que la información este de acuerdo a lo esperado.

In [13]:
df_director.head(15)

,id,title,direction_name,release_date,budget,revenue,return
0,0,Midnight Man,NaN,1997-08-20,0.0,0.0,0.00
1,1,Mardock Scramble: The Third Exhaust,NaN,2012-09-29,0.0,0.0,0.00
2,2,Ariel,Aki Kaurismäki,1988-10-21,0.0,0.0,0.00
3,3,Shadows in Paradise,Aki Kaurismäki,1986-10-16,0.0,0.0,0.00
4,4,Avalanche Sharks,NaN,2014-01-01,0.0,0.0,0.00
5,5,Four Rooms,"[Allison Anders, Alexandre Rockwell, Robert Ro...",1995-12-09,4000000.0,4300000.0,1.08
6,6,Judgment Night,Stephen Hopkins,1993-10-15,0.0,12136938.0,0.00
7,11,Star Wars,George Lucas,1977-05-25,11000000.0,775398007.0,70.49
8,12,Finding Nemo,Andrew Stanton,2003-05-30,94000000.0,940335536.0,10.00
9,13,Forrest Gump,Robert Zemeckis,1994-07-06,55000000.0,677945399.0,12.33


**9.** Se cambia el formato a la columna **`release date`** la cual debe tener el formato **`AAAA-mm-dd`**.

In [14]:
df_director['release_date']=pd.to_datetime(df_director['release_date'])

**10.** Se consulta el detallado de la información del DataFrame para constatar los cambios en el formato de fecha.

In [15]:
df_director.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45349 entries, 0 to 45348
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              45349 non-null  int64         
 1   title           45349 non-null  object        
 2   direction_name  45253 non-null  object        
 3   release_date    45349 non-null  datetime64[ns]
 4   budget          45349 non-null  float64       
 5   revenue         45349 non-null  float64       
 6   return          45349 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 2.8+ MB


**11.** Se cambian los nombres de las variables para efectos de mayor claridad al momento de la creación de las funciones para los **`endpoint`**.

In [16]:
new_names_columns = {'id': 'Id_Peliculas',
                    'title': 'Nombre_Pelicula',
                    'direction_name': 'Nombre_Director',
                    'release_date': 'Fecha_Estreno_Pelicula',
                    'budget': 'Presupuesto',
                    'revenue': 'Recaudacion_Total',
                    'return': 'Retorno_Inversion'}

df_director.rename(columns=new_names_columns, inplace=True)
df_director.head(20)

,Id_Peliculas,Nombre_Pelicula,Nombre_Director,Fecha_Estreno_Pelicula,Presupuesto,Recaudacion_Total,Retorno_Inversion
0,0,Midnight Man,NaN,1997-08-20,0.0,0.0,0.00
1,1,Mardock Scramble: The Third Exhaust,NaN,2012-09-29,0.0,0.0,0.00
2,2,Ariel,Aki Kaurismäki,1988-10-21,0.0,0.0,0.00
3,3,Shadows in Paradise,Aki Kaurismäki,1986-10-16,0.0,0.0,0.00
4,4,Avalanche Sharks,NaN,2014-01-01,0.0,0.0,0.00
5,5,Four Rooms,"[Allison Anders, Alexandre Rockwell, Robert Ro...",1995-12-09,4000000.0,4300000.0,1.08
6,6,Judgment Night,Stephen Hopkins,1993-10-15,0.0,12136938.0,0.00
7,11,Star Wars,George Lucas,1977-05-25,11000000.0,775398007.0,70.49
8,12,Finding Nemo,Andrew Stanton,2003-05-30,94000000.0,940335536.0,10.00
9,13,Forrest Gump,Robert Zemeckis,1994-07-06,55000000.0,677945399.0,12.33


# <h1 align=left>**`Load`**</h1>

**12.** Teniendo lista la información con los datos deseados se procede con la creación del nuevo Dataset.

In [17]:
df_director.to_csv("..\Data\directors_dataset.csv")